In [1]:
import time
import os
import shutil
import sys
import h5py
import re
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
import random                                 as rnd
from matplotlib                 import pyplot as plt 

print("="*50)
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [2]:
def write_predictiondata(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):
    
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]
                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [3]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
Dimension           = 'nondim'

# ExcitType           = 'KInel'
# NNRunIdx            = 9
ExcitType           = 'KExch'
NNRunIdx            = 6

if (Dimension == 'nondim'):
    PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures_nondim/' + ExcitType + '/' 
else:
    PathToRunFld        = SurQCTFldr + '/../' + RatesType + '_N3_TransLearn/all_temperatures_nondim/' + ExcitType + '/' 


TTranVec            = [10000.0]

Molecules           = ['N2','N2']
Atoms               = ['N','N']
System = 'N3_NASA'

# Molecules           = ['NO','NO']
# Atoms               = ['N','N']
# System = 'NON_UMN'

#Molecules           = ['CO','CO']
#Atoms               = ['O','O']
#System = 'CO2_NASA'

ZeroDRunIdx         = NNRunIdx

MinValueTrain       = 1.e-15

In [4]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...


In [5]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld 
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 
InputData.KineticFldr     = InputData.WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/'+Dimension+'_'+System+'_Active_Run'+str(ZeroDRunIdx)+'/'

# ## N2 from N3
if (Dimension == 'nondim'):
    InputData.PathToLevelsFile = [WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv',
                                  WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv']
    InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
    InputData.Molecules       = ['N2','N2'] 
    InputData.PathToDiatFile  = [WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
                                 WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   

InputData.QCTKineticFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D_semi/database/kinetics/'+System+'/'

# # NO
# InputData.PathToLevelsFile = [WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_log_nd.csv',
#                               WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_log_nd.csv']
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/NON_UMN.hdf5'
# InputData.Molecules       = ['NO','NO'] 
# InputData.PathToDiatFile  = [WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp',
#                              WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp']   
# InputData.QCTKineticFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D/database/kinetics/'+System+'/'

# # NO, N2
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/NON_UMN.hdf5'
# InputData.Molecules       = ['NO','N2'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_log_nd.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_UMN_ForN2O2_log_nd.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp',
#                              InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/UMN_ForN2O2/Recomputed.inp'] 

## For CO from COO PES
#InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/CO2_NASA.hdf5'
#InputData.Molecules       = ['CO','CO'] 
#InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/CO_NASA_log_nd.csv',
#                             InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/CO_NASA_log_nd.csv']
#InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat',
#                             InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat']



[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [6]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg = 0
InputData.TrainIntFlg       = 0
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['log_EVib_i', 'log_ERot_i', 'ri_i', 'log_rorMin_i', 'TTran_i', 'log_EVib_Delta', 'log_ERot_Delta', 'ri_Delta', 'log_rorMin_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/Params//ModelCheckpoint/cp-0831.ckpt


In [7]:
InputData.TransferFlg = True
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
# MinValueTrain      = 1.e-16 * InputData.MultFact
# MinValueTest       = 1.e-16 * InputData.MultFact
# NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

InputData.iLevelsVecTest = list(np.array(InputData.iLevelsVecTest) - 1)

[SurQCT]:   Reading Variables:  ['log_EVib', 'jqn', 'ri', 'log_ERot', 'vqn', 'log_rorMin']


In [8]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array(TTranVec), np.array(TTranVec))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp


In [9]:
#========================================================================================================
def read_kexcitdata_PLATO(InputData, PathToKineticFile, TTra):
    print(PathToKineticFile)
    Rates = np.zeros((NLevels[0],NLevels[0]))
    i2j_dict = {}
    with open(PathToKineticFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('N2|N|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            jIdx    = int(linestemp[8]);
            RateTemp = float(linestemp[12]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates, i2j_dict

In [10]:
if(ExcitType == 'KInel'): 
    PathToKineticFile  = InputData.QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Inel.dat'
elif(ExcitType == 'KExch'): 
    PathToKineticFile  = InputData.QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Exch_Type1.dat'
    
QCTRates, i2j_dict = read_kexcitdata_PLATO(InputData,PathToKineticFile,TTranVec[0])

/home/venturi/WORKSPACE//Air_Database/Run_0D_semi/database/kinetics/N3_NASA//T10000K/Inel.dat


# Get state number for states belonging to vibrational level v

In [10]:
# vqnVec_toWrite=np.array([0,1,2,3,4], dtype=int)
vqnVec_toWrite=np.linspace(5, 61, num=56, endpoint=False,dtype=int)
vqnVec = DiatData[0]['vqn'].to_numpy(int)

In [11]:
vqnVec

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
iDxVec=np.empty(0,dtype=int)

for i in range(np.size(vqnVec_toWrite)):    
    iDxVecTemp = [idx for idx in np.arange(NLevels[0]) if (vqnVec[idx] == vqnVec_toWrite[i])]
    iDxVec=np.append(iDxVec,iDxVecTemp)

print(iDxVec)

[ 285  286  287 ... 7428 7430 7431]


# Write surQCT Rates for given v

In [13]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass
    
    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'-'+str(np.max(vqnVec_toWrite))+'.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_v'+str(np.min(vqnVec_toWrite))+'-' + str(np.max(vqnVec_toWrite)) +'.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(iDxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/8046 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run6//T10000K/Exch_Type1_v5-60.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 8046/8046 [34:30<00:00,  3.89it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run6//T10000K/Exch_Type1_v5-60.dat


# Write QCT Rates for given vs

In [18]:
try:
    os.makedirs(InputData.QCTKineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.QCTKineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'-'+str(np.max(vqnVec_toWrite))+'.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_v'+str(np.min(vqnVec_toWrite))+'-' + str(np.max(vqnVec_toWrite)) +'.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(iDxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)

        if(ExcitType == 'KInel'):
            Maskj = (iDxVec < iIdx)
            jIdxVecQCT = iDxVec[Maskj]
            KInel_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
            csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KInel_NN_FWD)
        elif(ExcitType == 'KExch'):
            jIdxVecQCT = i2j_dict[iIdx]    
            KExch_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
            csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KExch_NN_FWD)
                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   3%|▎         | 72/2844 [00:00<00:03, 709.34it/s]

[SurQCT]:    TTran =  5000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_semi/database/kinetics/N3_NASA//T5000K/Inel_v0-10.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 2844/2844 [00:11<00:00, 249.03it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_semi/database/kinetics/N3_NASA//T5000K/Inel_v0-10.dat


# Replacing all rates for v = 0 and v = 1 with QCT ones 

In [13]:
def write_predictiondata_QCT(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec, KVecQCT):
    
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVecQCT[i] >= MinValueTrain and (vqnVec[jLevelVec[i]] == 0 or vqnVec[jLevelVec[i]]==1)):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVecQCT[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVecQCT[i]
                csvkinetics.write(Line)

            elif(KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]

                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [16]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass
    
    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'-'+str(np.max(vqnVec_toWrite))+'_v0v1QCT.dat'
        csvkinetics_KInel = write_predictiondata_QCT(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_v'+str(np.min(vqnVec_toWrite))+'-' + str(np.max(vqnVec_toWrite)) +'.dat'
        csvkinetics_KExch = write_predictiondata_QCT(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(iDxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        # Writing QCT Rates
        if((vqnVec[iIdx]==0 or vqnVec[iIdx]==1) and iIdx!=0):
            if(ExcitType == 'KInel'):
                jIdxVecQCT = i2j_dict[iIdx]
                KInel_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
                csvkinetics_KInel = write_predictiondata_QCT(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KInel_NN_FWD, KInel_NN_FWD)
            elif(ExcitType == 'KExch'):
                jIdxVecQCT = i2j_dict[iIdx+2]    
                KExch_NN_FWD = QCTRates[iIdx+2,jIdxVecQCT]
                csvkinetics_KExch = write_predictiondata_QCT(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx+2, jIdxVecQCT, KExch_NN_FWD, KExch_NN_FWD)

        # Writing NN Rates    
        else:
            
            iiIdxVec              = [iIdx]*jNLevels

            TTranVecTemp          = np.ones((jNLevels))*TTran
            TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
            TTranDataTemp.index   = jIdxVec


            iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
            iLevelsDataTemp.index = jIdxVec

            jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            if (OtherVar == '_Delta'):
                jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
            else:
                jLevelsDataTemp   = jLevelsDataTemp
            jLevelsDataTemp.index = jIdxVec

            kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            kLevelsDataTemp.index = jIdxVec


            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]

            xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)


            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                if(ExcitType == 'KInel'):
                    KInel_QCT_FWD = np.zeros((iIdx))
                    if iIdx in i2j_dict.keys():
                        jIdxVecQCT    = i2j_dict[iIdx]
                        KInel_QCT_FWD[jIdxVecQCT] = QCTRates[iIdx,jIdxVecQCT]
                    KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                    csvkinetics_KInel = write_predictiondata_QCT(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD, KInel_QCT_FWD)

                elif(ExcitType == 'KExch'):      
                    KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                    csvkinetics_KExch = write_predictiondata_QCT(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata_QCT(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata_QCT(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 1/2844 [00:00<07:13,  6.56it/s]

[SurQCT]:    TTran =  5000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run9//T5000K/Inel_v0-10_v0v1QCT.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 2844/2844 [07:47<00:00,  6.08it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run9//T5000K/Inel_v0-10_v0v1QCT.dat


# Replacing rates from v=0,1 going to v=0,1 only and vice versa

In [ ]:
def write_predictiondata_QCT2(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec, KVecQCT):
    
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVecQCT[i] >= MinValueTrain and vqnVec[iLevel]<=1 and (vqnVec[jLevelVec[i]] == 0 or vqnVec[jLevelVec[i]]==1)):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVecQCT[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVecQCT[i]
                csvkinetics.write(Line)

            elif(KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]

                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [ ]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass
    
    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'-'+str(np.max(vqnVec_toWrite))+'_v0v1-v0v1QCT.dat'
        csvkinetics_KInel = write_predictiondata_QCT2(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type2_v'+str(np.min(vqnVec_toWrite))+'-' + str(np.max(vqnVec_toWrite)) +'.dat'
        csvkinetics_KExch = write_predictiondata_QCT2(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(iDxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in iDxVec if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        # Writing QCT Rates
        if((vqnVec[iIdx]==0 or vqnVec[iIdx]==1) and iIdx!=0):
            if(ExcitType == 'KInel'):
                jIdxVecQCT = i2j_dict[iIdx]
                KInel_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
                csvkinetics_KInel = write_predictiondata_QCT2(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KInel_NN_FWD, KInel_NN_FWD)
            elif(ExcitType == 'KExch'):
                jIdxVecQCT = i2j_dict[iIdx+2]    
                KExch_NN_FWD = QCTRates[iIdx+2,jIdxVecQCT]
                csvkinetics_KExch = write_predictiondata_QCT2(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx+2, jIdxVecQCT, KExch_NN_FWD, KExch_NN_FWD)

        # Writing NN Rates    
        else:
            
            iiIdxVec              = [iIdx]*jNLevels

            TTranVecTemp          = np.ones((jNLevels))*TTran
            TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
            TTranDataTemp.index   = jIdxVec


            iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
            iLevelsDataTemp.index = jIdxVec

            jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            if (OtherVar == '_Delta'):
                jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
            else:
                jLevelsDataTemp   = jLevelsDataTemp
            jLevelsDataTemp.index = jIdxVec

            kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            kLevelsDataTemp.index = jIdxVec


            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]

            xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)


            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                if(ExcitType == 'KInel'):
                    KInel_QCT_FWD = np.zeros((iIdx))
                    if iIdx in i2j_dict.keys():
                        jIdxVecQCT    = i2j_dict[iIdx]
                        KInel_QCT_FWD[jIdxVecQCT] = QCTRates[iIdx,jIdxVecQCT]
                    KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                    csvkinetics_KInel = write_predictiondata_QCT2(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD, KInel_QCT_FWD)

                elif(ExcitType == 'KExch'):      
                    KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                    csvkinetics_KExch = write_predictiondata_QCT2(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata_QCT2(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata_QCT2(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None, None)


# Writing all rates and replacing specific vs with QCT

In [13]:
def write_predictiondata_QCT3(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec, KVecQCT):
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVecQCT[i] >= MinValueTrain and (vqnVec[jLevelVec[i]]==1)):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVecQCT[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVecQCT[i]
                csvkinetics.write(Line)

            elif(KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]

                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [14]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass
    
    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'QCT.dat'
        csvkinetics_KInel = write_predictiondata_QCT3(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_v'+str(np.min(vqnVec_toWrite))+'QCT.dat'
        csvkinetics_KExch = write_predictiondata_QCT3(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in range(NLevels[0]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in range(NLevels[0]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in range(NLevels[0]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in range(NLevels[0]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        # Writing QCT Rates
        if((vqnVec[iIdx]==1) and iIdx!=0):
            if(ExcitType == 'KInel'):
                jIdxVecQCT = i2j_dict[iIdx]
                KInel_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
                csvkinetics_KInel = write_predictiondata_QCT3(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KInel_NN_FWD, KInel_NN_FWD)
            elif(ExcitType == 'KExch'):
                jIdxVecQCT = i2j_dict[iIdx]    
                KExch_NN_FWD = QCTRates[iIdx,jIdxVecQCT]
                csvkinetics_KExch = write_predictiondata_QCT3(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVecQCT, KExch_NN_FWD, KExch_NN_FWD)

        # Writing NN Rates    
        else:
            iiIdxVec              = [iIdx]*jNLevels

            TTranVecTemp          = np.ones((jNLevels))*TTran
            TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
            TTranDataTemp.index   = jIdxVec


            iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
            iLevelsDataTemp.index = jIdxVec

            jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            if (OtherVar == '_Delta'):
                jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
            else:
                jLevelsDataTemp   = jLevelsDataTemp
            jLevelsDataTemp.index = jIdxVec

            kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            kLevelsDataTemp.index = jIdxVec


            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]

            xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)


            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                if(ExcitType == 'KInel'):
                    KInel_QCT_FWD = np.zeros((iIdx))
                    if iIdx in i2j_dict.keys():
                        jIdxVecQCT    = i2j_dict[iIdx]
                        KInel_QCT_FWD[jIdxVecQCT] = QCTRates[iIdx,jIdxVecQCT]
                    KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                    csvkinetics_KInel = write_predictiondata_QCT3(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD, KInel_QCT_FWD)

                elif(ExcitType == 'KExch'):      
                    KExch_QCT_FWD = np.zeros((iIdx))
                    if iIdx in i2j_dict.keys():
                        jIdxVecQCT    = i2j_dict[iIdx]
                        KExch_QCT_FWD[jIdxVecQCT] = QCTRates[iIdx,jIdxVecQCT]
                    KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                    csvkinetics_KExch = write_predictiondata_QCT3(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD, KExch_QCT_FWD)
                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata_QCT3(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata_QCT3(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/9399 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/transfer_N3_NASA_Active_Run2//T10000K/Inel_v1QCT.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 9399/9399 [59:55<00:00,  2.61it/s] 

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/transfer_N3_NASA_Active_Run2//T10000K/Inel_v1QCT.dat


# Writing some rates with transfer learned model and others with GenModel

In [ ]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass
    
    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_v'+str(np.min(vqnVec_toWrite))+'-'+str(np.max(vqnVec_toWrite))+'.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_v'+str(np.min(vqnVec_toWrite))+'-' + str(np.max(vqnVec_toWrite)) +'.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(iDxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in range(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in range(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in range(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in range(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/1353 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/transfer_N3_NASA_Active_Run4//T10000K/Exch_Type1_v0-4.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:  99%|█████████▉| 1341/1353 [10:45<00:06,  1.76it/s]